In [2]:
import pandas as pd
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Flatten, Dense
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Define lists to store the feature and target data
X = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'frameTime','Valence_Value'], axis=1).values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define a custom learning rate schedule
class CustomLearningRateSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, decay_rate):
        super(CustomLearningRateSchedule, self).__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate

    def __call__(self, step):
        learning_rate = self.initial_learning_rate * self.decay_rate ** (step / self.decay_steps)
        return learning_rate

# Create a custom learning rate schedule
lr_schedule = CustomLearningRateSchedule(initial_learning_rate=1e-4, decay_steps=10, decay_rate=0.1)

# Build the BiLSTM model for Arousal
model_v = Sequential()
model_v.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_reshaped.shape[1], 1)))
model_v.add(Flatten())
model_v.add(Dense(1))
model_v.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

# Train the Arousal model
model_v.fit(X_reshaped, y_v, epochs=55, batch_size=32)

# Select the optimal learning rate
optimal_lr = lr_schedule(0)  # Evaluate the learning rate at the first step
print(f'Optimal Learning Rate: {optimal_lr}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/55
3317/3317 [==============================] - 65s 17ms/step - loss: 0.0544
Epoch 2/55
3317/3317 [==============================] - 60s 18ms/step - loss: 0.0544
Epoch 3/55
3317/3317 [==============================] - 60s 18ms/step - loss: 0.0544
Epoch 4/55
3317/3317 [==============================] - 60s 18ms/step - loss: 0.0544
Epoch 5/55
3317/3317 [==============================] - 60s 18ms/step - loss: 0.0544
Epoch 6/55
3317/3317 [==============================] - 57s 17ms/step - loss: 0.0544
Epoch 7/55
3317/3317 [==============================] - 57s 17ms/step - loss: 0.0544
Epoch 8/55
3317/3317 [==============================] - 58s 17ms/step - loss: 0.0544
Epoch 9/55
3317/3317 [==============================] - 58s 17ms/step - loss: 0.0544
Epoch 10/55
3317/3317 [==============================] - 58s 17ms/step - loss: 0.0544
Epoch 11/55
3317/331